In [31]:
import numpy as np
import pandas as pd
from pathlib import Path
from numpy import linalg
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

In [32]:

# Load the data using the provided file path
file_path = r"C:\Users\lenovo\Desktop\Uni STuff\SFU\Year 4\MSE 491\Calc\MSE_413\Lab3\code\Data\Human activity semi-supervised.csv"
df = pd.read_csv(file_path)

# Make copies and perform necessary operations
Data_unlabeled = df.copy()  # Create a copy for unlabeled data
Data_unlabeled2 = Data_unlabeled.dropna()  # Drop rows with missing values
Data_labeled = Data_unlabeled.copy()  # Create a copy for labeled data
Data_train = Data_unlabeled.drop(["subject", "Activity"], axis=1)  # Remove 'subject' and 'Activity' columns from the feature set

C:\Users\lenovo\AppData\Local\Temp\ipykernel_15104\838331052.py:3: DtypeWarning: Columns (562) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [33]:
# Perform K-means clustering
n_clusters = 9
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init="auto").fit(Data_train)
kmeans_2 = kmeans.labels_
centers = kmeans.cluster_centers_

In [34]:
import numpy as np

# Calculate distances and assign labels
Features = Data_unlabeled2.drop(["subject", "Activity"], axis=1)
Feature = Features.to_numpy()

# Calculate the number of rows in the feature matrix
num_rows, num_cols = Feature.shape

# Initialize list to store distances
dist = [None] * num_rows

# Loop over clusters
for i in range(n_clusters):
    # Loop over data points
    for j in range(num_rows):
        # Calculate Euclidean distance between centroid and data point
        dist[j] = np.linalg.norm(centers[i] - Feature[j])

    # Find the index of the closest data point to the centroid
    index_min = np.argmin(dist)

    # Assign labels to data points in the same cluster
    for k in range(num_rows):
        if kmeans_2[k] == i:
            # Update labeled data with the corresponding activity from the closest data point
            Data_labeled.loc[k, "Activity"] = Data_unlabeled.loc[Features.index[index_min], "Activity"]


In [35]:
#Task 4: Train a random forest classifier on labeled dataset using K-means for 5<K<13
results = {}
for k in range(5, 13):
    kmeans = KMeans(n_clusters=k, random_state=42, n_init="auto").fit(Data_train)
    kmeans_labels = kmeans.labels_
    rf_classifier = RandomForestClassifier(n_estimators=10, max_depth=10, random_state=42)
    rf_classifier.fit(Data_train, kmeans_labels)
    # Predict labels for the training set (using the same set for illustration purposes)
    y_train_pred = rf_classifier.predict(Data_train)
    accuracy = accuracy_score(kmeans_labels, y_train_pred)
    results[k] = accuracy

best_k = max(results, key=results.get)
best_accuracy = results[best_k]

print("Best K for K-means:", best_k)
print("Accuracy with best K:", best_accuracy)

Best K for K-means: 6
Accuracy with best K: 0.9956474428726877


In [37]:
# Task 5: Label unlabeled data based on closest activity to centroid
for i in range(n_clusters):
    # Initialize variables to store the minimum distance and corresponding activity
    min_distance = float('inf')  # Initialize with a large value
    closest_activity = None

    # Calculate distances between each data point and centroid of cluster i
    for j in range(len(Feature)):
        distance = np.linalg.norm(centers[i] - Feature[j])

        # Check if the data point is unlabeled and if its distance is smaller than the current minimum
        if kmeans_2[j] == i and pd.isnull(Data_labeled.iloc[j]["Activity"]) and distance < min_distance:
            min_distance = distance
            closest_activity = Data_unlabeled2.iloc[j]["Activity"]

    # Label all unlabeled data points in cluster i with the closest activity
    for k in range(len(Feature)):
        if kmeans_2[k] == i and pd.isnull(Data_labeled.iloc[k]["Activity"]):
            Data_labeled.loc[k, "Activity"] = closest_activity

In [22]:
# Task 6: Train random forest classifier using K-means for 5 < K < 13

# Initialize dictionary to store trained models for different K
trained_models = {}

# Define parameters for random forest classifier
n_estimators = 10
max_depth = 10
random_state = 42

# Iterate over values of K from 5 to 12
for k in range(5, 13):
    # Perform K-means clustering
    kmeans = KMeans(n_clusters=k, random_state=42, n_init="auto").fit(Data_train)
    kmeans_labels = kmeans.labels_
    # Train random forest classifier
    rf_classifier = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=random_state)
    rf_classifier.fit(Data_train, kmeans_labels)
    # Store the trained model for evaluation
    trained_models[k] = rf_classifier
# Print the trained models
print("Trained models for different values of K:", trained_models)

Trained models for different values of K: {5: RandomForestClassifier(max_depth=10, n_estimators=10, random_state=42), 6: RandomForestClassifier(max_depth=10, n_estimators=10, random_state=42), 7: RandomForestClassifier(max_depth=10, n_estimators=10, random_state=42), 8: RandomForestClassifier(max_depth=10, n_estimators=10, random_state=42), 9: RandomForestClassifier(max_depth=10, n_estimators=10, random_state=42), 10: RandomForestClassifier(max_depth=10, n_estimators=10, random_state=42), 11: RandomForestClassifier(max_depth=10, n_estimators=10, random_state=42), 12: RandomForestClassifier(max_depth=10, n_estimators=10, random_state=42)}
